Descripción del proyecto

Este notebook implementa la planificación y simulación de una trayectoria lineal para un robot SCARA de configuración RRP utilizando Robotics Toolbox for Python.
Se define el modelo cinemático mediante parámetros Denavit–Hartenberg (DH) y se genera una trayectoria cartesiana entre tres puntos 
P1
, 
P2
P
2
	​

 y 
P3
P
3
	​

. Posteriormente, se calcula la cinemática inversa para obtener las configuraciones articulares correspondientes y se crea una animación en formato GIF, donde se observa el movimiento del robot siguiendo la trayectoria lineal establecida.

El objetivo principal es validar la capacidad del robot para desplazarse suavemente entre posiciones específicas dentro de su espacio de trabajo, cumpliendo con los requisitos de una aplicación de ensamble o manipulación ligera.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import roboticstoolbox as rtb
from spatialmath import SE3
from IPython.display import Image, display

# ---------- 1. Definición del robot SCARA RRP ----------
mm = 1e-3  # milímetros -> metros

# Base 90 mm, L1 80 mm, L2 80 mm, stroke ≈ 30 mm
L1 = rtb.RevoluteDH(d=90 * mm, a=80 * mm, alpha=0)
L2 = rtb.RevoluteDH(d=0,       a=80 * mm, alpha=np.pi)
L3 = rtb.PrismaticDH(theta=0,  a=0,       alpha=0, qlim=[0, 30 * mm])

robot = rtb.DHRobot([L1, L2, L3], name="SCARA_Martinez")
print(robot)

# ---------- 2. Definir P1, P2, P3 (en METROS, bien separados) ----------
# Elegimos puntos dentro del radio ≈ 0.16 m, pero con CAMBIOS GRANDES
P1 = SE3(0.10, 0.05, 0.030)   # más cerca del centro
P2 = SE3(0.16, 0.00, 0.030)   # casi totalmente extendido en X
P3 = SE3(0.05, 0.13, 0.020)   # hacia "arriba" en Y y más abajo en Z

print("P1:", P1)
print("P2:", P2)
print("P3:", P3)

# ---------- 3. Trayectorias lineales en espacio cartesiano ----------
steps12 = 40
steps23 = 40

traj_12 = rtb.ctraj(P1, P2, steps12)   # SE3Array
traj_23 = rtb.ctraj(P2, P3, steps23)   # SE3Array

# Lista de SE3 concatenada (sin repetir P2)
trayectoria = list(traj_12) + list(traj_23[1:])
print(f"Total de puntos en trayectoria: {len(trayectoria)}")

# ---------- 4. Cinemática inversa punto a punto ----------
mask_xyz = [1, 1, 1, 0, 0, 0]

q_list = []
q_actual = np.array([0.0, 0.0, 0.02])  # postura inicial aproximada

for i, T in enumerate(trayectoria):
    sol = robot.ikine_LM(T, q0=q_actual, mask=mask_xyz)
    if sol.success:
        q_actual = sol.q
        q_list.append(q_actual)
    else:
        print(f"Advertencia: IK falló en el paso {i}, se repite q anterior")
        q_list.append(q_actual)

q_list = np.array(q_list)
print("Configuraciones articulares calculadas:", q_list.shape)
print("Primeras q:", q_list[0])
print("Últimas q:", q_list[-1])

# ---------- 5. Animación automática y generación de GIF ----------
gif_name = "scara_trayectoria_lineal.gif"

env = robot.plot(
    q_list,
    backend='pyplot',
    movie=gif_name,   # <<--- genera el GIF automáticamente
    block=False,
    dt=0.05           # tiempo entre cuadros (más grande = movimiento más visible)
)

# Ajuste de ejes (se aplica al último plot, pero no afecta al GIF ya guardado)
env.ax.set_xlim([-0.10, 0.25])
env.ax.set_ylim([-0.15, 0.20])
env.ax.set_zlim([0.00, 0.15])
env.ax.set_xlabel('X (m)')
env.ax.set_ylabel('Y (m)')
env.ax.set_zlabel('Z (m)')
env.ax.set_title('SCARA RRP - Trayectoria lineal P1 → P2 → P3')

print(f"\nSi no hubo errores, el GIF se guardó como: {gif_name}")

# ---------- 6. Mostrar el GIF en el notebook ----------
try:
    display(Image(filename=gif_name))
except Exception as e:
    print("No se pudo mostrar el GIF, pero debe estar en la carpeta del notebook.")
    print(e)

plt.close('all')
